<a href="https://colab.research.google.com/github/Ankur3107/large-scale-multi-label-classification/blob/master/large_scale_multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
!wget -O datasets.zip http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/datasets.zip


--2020-08-23 11:37:34--  http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/datasets.zip
Resolving nlp.cs.aueb.gr (nlp.cs.aueb.gr)... 195.251.248.252
Connecting to nlp.cs.aueb.gr (nlp.cs.aueb.gr)|195.251.248.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135996905 (130M) [application/zip]
Saving to: ‘datasets.zip’

datasets.zip        100%[===================>] 129.70M  2.29MB/s    in 2m 12s  

2020-08-23 11:39:48 (1006 KB/s) - ‘datasets.zip’ saved [135996905/135996905]



In [ ]:
!unzip datasets.zip -d EURLEX57K

Streaming output truncated to the last 5000 lines.
  inflating: EURLEX57K/dataset/dev/32004R1662.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._32004R1662.json  
  inflating: EURLEX57K/dataset/dev/32004R0970.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._32004R0970.json  
  inflating: EURLEX57K/dataset/dev/31999R0014.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._31999R0014.json  
  inflating: EURLEX57K/dataset/dev/32007R1267.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._32007R1267.json  
  inflating: EURLEX57K/dataset/dev/31987R3625.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._31987R3625.json  
  inflating: EURLEX57K/dataset/dev/31999R2183.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._31999R2183.json  
  inflating: EURLEX57K/dataset/dev/32012R0997.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._32012R0997.json  
  inflating: EURLEX57K/dataset/dev/31994R2926.json  
  inflating: EURLEX57K/__MACOSX/dataset/dev/._31994R2926.json  
  inflating: 

In [ ]:
cat EURLEX57K/dataset/dev/31995D0380.json

{"celex_id": "31995D0380", "uri": "http://publications.europa.eu/resource/cellar/1ea4956f-28c2-4193-a15d-352d59bdfd49", "type": "Decision", "concepts": ["1895", "2711", "4057", "4257", "5962"], "title": "95/380/EC: Commission Decision of 18 September 1995 amending Commission Decisions 94/432/EC, 94/433/EC and 94/434/EC laying down detailed rules for the application of Council Directives 93/23/EEC on the statistical surveys to be carried out on pig production, 93/24/EEC on the statistical surveys to be carried out on bovine animal production and 93/25/EEC on the statistical surveys to be carried out on sheep and goat stocks\n", "header": "COMMISSION DECISION  of 18 September 1995 amending Commission Decisions 94/432/EC, 94/433/EC and 94/434/EC laying down  detailed rules for the application of Council Directives 93/23/EEC on the statistical surveys to be  carried out on pig production, 93/24/EEC on the statistical surveys to be carried out on bovine  animal production and 93/25/EEC on t

In [ ]:
import json

In [ ]:
with open('EURLEX57K/dataset/dev/31995D0380.json') as file:
  data = json.load(file)

In [ ]:
data

{'attachments': 'Done at Brussels, 18 September 1995.\nFor the Commission Yves-Thibault DE SILGUY Member of the Commission',
 'celex_id': '31995D0380',
 'concepts': ['1895', '2711', '4057', '4257', '5962'],
 'header': 'COMMISSION DECISION  of 18 September 1995 amending Commission Decisions 94/432/EC, 94/433/EC and 94/434/EC laying down  detailed rules for the application of Council Directives 93/23/EEC on the statistical surveys to be  carried out on pig production, 93/24/EEC on the statistical surveys to be carried out on bovine  animal production and 93/25/EEC on the statistical surveys to be carried out on sheep and goat  stocks (Text with EEA relevance) (95/380/EC)\nTHE COMMISSION OF THE EUROPEAN  COMMUNITIES',
 'main_body': ["Decision 94/432/EC laying down detailed rules for the application of  Directive 93/23/EEC shall be amended as follows:\n1.  Annex I shall be supplemented with the following text:\n'Austria: Bundeslaender Finland: Etelae-Suomi Sisae-Suomi Pohjanmaa Pohjois-Suo

In [ ]:
!rm datasets.zip
!rm -rf EURLEX57K/__MACOSX
!mv EURLEX57K/dataset/* EURLEX57K/
!rm -rf EURLEX57K/dataset
!wget -O EURLEX57K/EURLEX57K.json http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/eurovoc_en.json

--2020-08-23 11:40:53--  http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/eurovoc_en.json
Resolving nlp.cs.aueb.gr (nlp.cs.aueb.gr)... 195.251.248.252
Connecting to nlp.cs.aueb.gr (nlp.cs.aueb.gr)|195.251.248.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921898 (900K) [text/plain]
Saving to: ‘EURLEX57K/EURLEX57K.json’

EURLEX57K/EURLEX57K 100%[===================>] 900.29K   159KB/s    in 5.7s    

2020-08-23 11:40:59 (159 KB/s) - ‘EURLEX57K/EURLEX57K.json’ saved [921898/921898]



In [ ]:
import glob, os
from collections import Counter
import tqdm, json

In [ ]:
DATA_SET_DIR = './'

In [ ]:
train_files = glob.glob(os.path.join(DATA_SET_DIR, 'EURLEX57K', 'train', '*.json'))

In [ ]:
documents = []
labels = []

for filename in tqdm.tqdm(train_files):
  with open(filename) as file:
      data = json.load(file)
      labels.append(data['concepts'])
      documents.append(data['main_body'])

100%|██████████| 45000/45000 [00:03<00:00, 12726.01it/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'text':documents,'label':labels})

In [ ]:
df.head()

,text,label
0,[The Annex to Regulation (EEC) No 3846/87 is h...,"[2068, 2069, 2734, 3568, 4381]"
1,[Third States’ contributions\n1. The contrib...,"[2084, 5556, 5744, 5889, 922]"
2,[Decision 2010/96/CFSP is hereby amended as fo...,"[218, 4212, 5610, 6927, 8482]"
3,[Mr Mario MINOJA is hereby appointed a member ...,"[1519, 3559, 6054]"
4,[The representative prices and additional duti...,"[2635, 2733, 3191, 4080]"


In [ ]:
df['flatten_text'] = df['text'].apply(lambda x: ' '.join(x))

In [ ]:
df.head()

,text,label,flatten_text
0,[The Annex to Regulation (EEC) No 3846/87 is h...,"[2068, 2069, 2734, 3568, 4381]",The Annex to Regulation (EEC) No 3846/87 is he...
1,[Third States’ contributions\n1. The contrib...,"[2084, 5556, 5744, 5889, 922]",Third States’ contributions\n1. The contribu...
2,[Decision 2010/96/CFSP is hereby amended as fo...,"[218, 4212, 5610, 6927, 8482]",Decision 2010/96/CFSP is hereby amended as fol...
3,[Mr Mario MINOJA is hereby appointed a member ...,"[1519, 3559, 6054]",Mr Mario MINOJA is hereby appointed a member o...
4,[The representative prices and additional duti...,"[2635, 2733, 3191, 4080]",The representative prices and additional dutie...


In [ ]:
df_new = df.explode('label')

In [ ]:
df_new.shape

(228322, 3)

In [ ]:
concept_df = pd.DataFrame({'concept_id':df_new.label.value_counts().index,'freq':df_new.label.value_counts().values})

In [ ]:
concept_df = concept_df[concept_df.freq>2000]

In [ ]:
concept_df.shape

(11, 2)

In [ ]:
concept_df.concept_id.values

array(['1309', '3568', '1118', '1605', '693', '2635', '20', '161', '2300',
       '1644', '2771'], dtype=object)

In [ ]:
df_new = df_new[df_new.label.isin(concept_df.concept_id.values)]

In [ ]:
df_new.shape

(30684, 3)

In [ ]:
df_new['id'] = df_new.index

In [ ]:
df_new = df_new[['id', 'flatten_text', 'label']]
df_new.head()

,id,flatten_text,label
0,0,The Annex to Regulation (EEC) No 3846/87 is he...,3568
4,4,The representative prices and additional dutie...,2635
7,7,1. Import licences applied for by traditiona...,1309
7,7,1. Import licences applied for by traditiona...,161
7,7,1. Import licences applied for by traditiona...,1644


In [ ]:
print('1. Unique ids :', len(df_new.id.unique()))
df_new_transformed = df_new.groupby(['id','label','flatten_text']).size().reset_index(name='count')
print('2. Unique ids :', len(df_new_transformed.id.unique()))
df_new_transformed['value'] = 1
df_new_transformed_temp = df_new_transformed.pivot(index=df_new_transformed.id, columns='label')['value'].fillna(0)

df_new_transformed_temp['id'] = df_new_transformed_temp.index
df_new_transformed_temp = df_new_transformed_temp.reset_index(drop=True)

1. Unique ids : 16310
2. Unique ids : 16310


In [ ]:
df_new_transformed_temp.columns, df_new_transformed_temp.shape

(Index(['1118', '1309', '1605', '161', '1644', '20', '2300', '2635', '2771',
        '3568', '693', 'id'],
       dtype='object', name='label'), (16310, 12))

In [ ]:
text_id_df = df_new.drop_duplicates(['id'])[['id','flatten_text']]
final_df = text_id_df.merge(df_new_transformed_temp, on='id')

In [ ]:
final_df.shape

(16310, 13)

In [ ]:
final_df.head()

,id,flatten_text,1118,1309,1605,161,1644,20,2300,2635,2771,3568,693
0,0,The Annex to Regulation (EEC) No 3846/87 is he...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4,The representative prices and additional dutie...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,7,1. Import licences applied for by traditiona...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,"The ""Direction des Services de l'Agriculture: ...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,16,The import duties in the rice sector referred ...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
final_df.columns

Index(['id', 'flatten_text', '1118', '1309', '1605', '161', '1644', '20',
       '2300', '2635', '2771', '3568', '693'],
      dtype='object')

In [ ]:
x_text = final_df.flatten_text.values.tolist()
y_label = final_df[['1118', '1309', '1605', '161', '1644', '20',
       '2300', '2635', '2771', '3568', '693']].to_numpy()

In [ ]:
y_label.shape

(16310, 11)

In [ ]:
text_vectorizer = preprocessing.TextVectorization(output_mode="int")
text_vectorizer.adapt(x_text)

In [ ]:
vocab = text_vectorizer.get_vocabulary()
len(vocab)

33103

In [ ]:
!pip install model-x

In [ ]:
from model_X.bilstm_architectures import *

In [ ]:
tf.keras.backend.clear_session()
inputs = tf.keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = BiLSTMGRUAttention(nb_words=len(vocab), embedding_size=64, is_embedding_trainable=True,attention_type='ScaledDotProductAttention' ,h_lstm=64, h_gru=32)(x)
outputs = tf.keras.layers.Dense(11, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)
model.compile('adam','binary_crossentropy','accuracy')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
text_vectorization (TextVectori (None, None)         0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     2118592     text_vectorization[0][0]         
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, None, 64)     0           embedding[0][0]                  
_______________________________________________________________________________________

In [ ]:
x_text = np.array(x_text)
x_text.shape

(16310,)

In [ ]:
model.fit(x_text, y_label, epochs=3, batch_size=16, validation_split=0.1)

Epoch 1/3
918/918 [==============================] - 384s 418ms/step - loss: 0.2919 - accuracy: 0.2911 - val_loss: 0.2084 - val_accuracy: 0.4960
Epoch 2/3
918/918 [==============================] - 382s 416ms/step - loss: 0.1823 - accuracy: 0.5209 - val_loss: 0.1554 - val_accuracy: 0.5365
Epoch 3/3
466/918 [==============>...............] - ETA: 3:04 - loss: 0.1428 - accuracy: 0.5534